In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive
!mkdir -p assignment_2
%cd assignment_2

/content/drive/MyDrive
/content/drive/MyDrive/assignment_2


In [3]:
%cd /content/assignment_2
!ls

/content/assignment_2
checkpoints  data  outputs  splits  src


In [4]:
!pip -q install ultralytics pycocotools lxml opencv-python torchmetrics==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.8 MB/s eta 0:00:00


In [5]:
!nvidia-smi

Fri Feb 20 15:56:18 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
%%writefile src/utils.py
import json
import random
import time
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List

import numpy as np
import torch

# ---- Reproducibility ----
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# ---- AverageMeter (like sgrvinod's training loop style) ----
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1):
        self.val = float(val)
        self.sum += float(val) * n
        self.count += n
        self.avg = self.sum / max(self.count, 1)

# ---- Timing context manager ----
class Timer:
    def __enter__(self):
        self.t0 = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc, tb):
        self.t1 = time.perf_counter()
        self.seconds = self.t1 - self.t0

# ---- 70/15/15 split ----
def split_indices(n: int, train: float, val: float, test: float, seed: int) -> Dict[str, List[int]]:
    assert abs((train + val + test) - 1.0) < 1e-6
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)

    n_train = int(n * train)
    n_val = int(n * val)

    return {
        "train": idx[:n_train].tolist(),
        "val": idx[n_train:n_train+n_val].tolist(),
        "test": idx[n_train+n_val:].tolist(),
    }

def save_json(obj: Any, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2))

def load_json(path: Path) -> Any:
    return json.loads(path.read_text())

# ---- Checkpointing (sgrvinod-style) ----
def save_checkpoint(path: Path, epoch: int, model: torch.nn.Module, optimizer: torch.optim.Optimizer, extra: Dict[str, Any] | None = None):
    path.parent.mkdir(parents=True, exist_ok=True)
    payload = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
    }
    if extra:
        payload.update(extra)
    torch.save(payload, path)

Writing src/utils.py


In [7]:
%%writefile src/config.py
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Config:
    # Project paths (assignment2/)
    root: Path = Path(".")
    data_dir: Path = root / "data"
    outputs_dir: Path = root / "outputs"
    ckpt_dir: Path = root / "checkpoints"
    splits_dir: Path = root / "splits"

    # Assignment requirements
    img_size: int = 512
    split_train: float = 0.70
    split_val: float = 0.15
    split_test: float = 0.15

    # Batch sizes (as recommended)
    frcnn_batch_size: int = 2     # 2–4
    yolo_batch_size: int = 8      # 8–16
    num_workers: int = 0

    # Epochs (within required ranges)
    penn_epochs: int = 12         # 10–15
    pets_epochs: int = 18         # 15–20

    # Early stopping
    patience: int = 3

    # Seed
    seed: int = 42

Writing src/config.py


In [8]:
%%writefile src/model.py
import torch
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def build_frcnn(num_classes: int) -> torch.nn.Module:
    """
    Assignment-required:
    torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
    Modern API uses weights="DEFAULT" but it's the same pretrained transfer learning requirement.
    """
    model = fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

Writing src/model.py


In [9]:
%%writefile src/datasets.py
from __future__ import annotations
from pathlib import Path
from typing import List, Tuple
import tarfile
import urllib.request
import zipfile

import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
from lxml import etree
import torchvision.transforms.functional as F


def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)


# ---------------------------
# Penn-Fudan (manual download)
# ---------------------------
PENNFUDAN_URL = "https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip"

def prepare_pennfudan(root: Path) -> Path:
    root.mkdir(parents=True, exist_ok=True)
    zip_path = root / "PennFudanPed.zip"
    data_dir = root / "PennFudanPed"
    if not data_dir.exists():
        if not zip_path.exists():
            urllib.request.urlretrieve(PENNFUDAN_URL, zip_path)
        with zipfile.ZipFile(zip_path, "r") as z:
            z.extractall(root)
    return data_dir

def masks_to_boxes(mask: torch.Tensor) -> torch.Tensor:
    # mask: [H,W] int, 0 background, 1..N instance ids
    obj_ids = torch.unique(mask)
    obj_ids = obj_ids[obj_ids != 0]

    boxes = []
    for oid in obj_ids:
        ys, xs = torch.where(mask == oid)
        if ys.numel() == 0:
            continue
        x1 = xs.min().item()
        y1 = ys.min().item()
        x2 = xs.max().item()
        y2 = ys.max().item()
        boxes.append([x1, y1, x2, y2])

    if len(boxes) == 0:
        return torch.zeros((0, 4), dtype=torch.float32)
    return torch.tensor(boxes, dtype=torch.float32)

class PennFudanDetection(Dataset):
    def __init__(self, root: str, img_size: int = 512):
        self.root = Path(root)
        data_dir = prepare_pennfudan(self.root)

        self.img_dir = data_dir / "PNGImages"
        self.mask_dir = data_dir / "PedMasks"

        self.images = sorted(self.img_dir.glob("*.png"))
        self.masks = sorted(self.mask_dir.glob("*.png"))
        assert len(self.images) == len(self.masks)

        self.img_size = img_size

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx: int):
        img_path = self.images[idx]
        mask_path = self.masks[idx]

        img_pil = Image.open(img_path).convert("RGB")
        mask_pil = Image.open(mask_path)

        w, h = img_pil.size

        # image -> tensor
        img = F.pil_to_tensor(img_pil).float() / 255.0  # [3,H,W]

        # mask -> numpy -> tensor (NO None anywhere)
        mask_np = np.array(mask_pil, dtype=np.int64)     # [H,W]
        mask = torch.from_numpy(mask_np)                 # int64

        boxes = masks_to_boxes(mask)
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)  # class 1 = person

        # resize image
        img = F.resize(img, [self.img_size, self.img_size])

        # scale boxes
        sx = self.img_size / w
        sy = self.img_size / h
        if boxes.numel() > 0:
            boxes[:, [0, 2]] *= sx
            boxes[:, [1, 3]] *= sy

        target = {"boxes": boxes, "labels": labels}
        return img, target


# ---------------------------
# Oxford-IIIT Pets subset
# ---------------------------
PETS_IMG_URL = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz"
PETS_ANN_URL = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz"

def _download_and_extract(url: str, out_dir: Path) -> None:
    out_dir.mkdir(parents=True, exist_ok=True)
    fname = out_dir / url.split("/")[-1]
    if not fname.exists():
        urllib.request.urlretrieve(url, fname)
    with tarfile.open(fname, "r:gz") as tar:
        tar.extractall(path=out_dir)

def prepare_pets(root: Path) -> Tuple[Path, Path]:
    _download_and_extract(PETS_IMG_URL, root)
    _download_and_extract(PETS_ANN_URL, root)
    return root / "images", root / "annotations" / "xmls"

def parse_bbox(xml_path: Path) -> Tuple[int, int, int, int]:
    tree = etree.parse(str(xml_path))
    bbox = tree.find(".//bndbox")
    xmin = int(bbox.findtext("xmin"))
    ymin = int(bbox.findtext("ymin"))
    xmax = int(bbox.findtext("xmax"))
    ymax = int(bbox.findtext("ymax"))
    return xmin, ymin, xmax, ymax

class PetsSubsetDetection(Dataset):
    def __init__(self, root: str, breeds: List[str], img_size: int = 512):
        self.root = Path(root)
        images_dir, xmls_dir = prepare_pets(self.root)

        self.items = []
        for jpg in images_dir.glob("*.jpg"):
            stem = jpg.stem
            breed_prefix = "_".join(stem.split("_")[:-1])
            if breed_prefix in breeds:
                xml = xmls_dir / f"{stem}.xml"
                if xml.exists():
                    self.items.append((jpg, xml, breed_prefix))

        self.breeds = breeds
        self.label_map = {b: i + 1 for i, b in enumerate(breeds)}
        self.img_size = img_size

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx: int):
        jpg_path, xml_path, breed = self.items[idx]
        img_pil = Image.open(jpg_path).convert("RGB")
        w, h = img_pil.size

        img = F.pil_to_tensor(img_pil).float() / 255.0
        img = F.resize(img, [self.img_size, self.img_size])

        xmin, ymin, xmax, ymax = parse_bbox(xml_path)
        sx = self.img_size / w
        sy = self.img_size / h

        box = torch.tensor([[xmin*sx, ymin*sy, xmax*sx, ymax*sy]], dtype=torch.float32)
        label = torch.tensor([self.label_map[breed]], dtype=torch.int64)

        target = {"boxes": box, "labels": label}
        return img, target

Writing src/datasets.py


In [10]:
%%writefile src/metrics.py
from typing import Dict, List, Tuple
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision

def _iou_xyxy(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    # a: [N,4], b: [M,4], xyxy
    N, M = a.size(0), b.size(0)
    if N == 0 or M == 0:
        return torch.zeros((N, M), device=a.device)

    ax1, ay1, ax2, ay2 = a[:, 0:1], a[:, 1:2], a[:, 2:3], a[:, 3:4]
    bx1, by1, bx2, by2 = b[:, 0], b[:, 1], b[:, 2], b[:, 3]

    ix1 = torch.maximum(ax1, bx1)
    iy1 = torch.maximum(ay1, by1)
    ix2 = torch.minimum(ax2, bx2)
    iy2 = torch.minimum(ay2, by2)

    iw = torch.clamp(ix2 - ix1, min=0)
    ih = torch.clamp(iy2 - iy1, min=0)
    inter = iw * ih

    area_a = torch.clamp(ax2 - ax1, min=0) * torch.clamp(ay2 - ay1, min=0)
    area_b = torch.clamp(bx2 - bx1, min=0) * torch.clamp(by2 - by1, min=0)
    union = area_a + area_b - inter
    return inter / torch.clamp(union, min=1e-6)

@torch.inference_mode()
def map50(preds: List[Dict[str, torch.Tensor]], targets: List[Dict[str, torch.Tensor]]) -> float:
    metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=[0.5])
    metric.update(preds, targets)
    out = metric.compute()
    return float(out["map"].item())  # with iou_thresholds=[0.5], this is mAP@0.5

def precision_recall_iou50(preds, targets, score_thresh: float = 0.5) -> Tuple[float, float]:
    tp = fp = fn = 0
    for p, t in zip(preds, targets):
        p_boxes = p["boxes"]
        p_scores = p["scores"]
        p_labels = p["labels"]

        keep = p_scores >= score_thresh
        p_boxes = p_boxes[keep]
        p_labels = p_labels[keep]
        p_scores = p_scores[keep]

        if p_scores.numel() > 0:
            order = torch.argsort(p_scores, descending=True)
            p_boxes = p_boxes[order]
            p_labels = p_labels[order]

        t_boxes = t["boxes"]
        t_labels = t["labels"]
        matched = set()

        for i in range(p_boxes.size(0)):
            same = (t_labels == p_labels[i]).nonzero(as_tuple=False).view(-1)
            same = [j.item() for j in same if j.item() not in matched]
            if not same:
                fp += 1
                continue
            ious = _iou_xyxy(p_boxes[i].unsqueeze(0), t_boxes[same]).squeeze(0)
            best = torch.argmax(ious).item()
            if ious[best].item() >= 0.5:
                tp += 1
                matched.add(same[best])
            else:
                fp += 1

        fn += (t_boxes.size(0) - len(matched))

    precision = tp / (tp + fp + 1e-9)
    recall = tp / (tp + fn + 1e-9)
    return float(precision), float(recall)

Writing src/metrics.py


In [11]:
%%writefile src/train.py
from typing import Dict, List, Tuple
import torch

from src.utils import AverageMeter, Timer, save_checkpoint

def train_one_epoch(model, loader, optimizer, device, scaler=None) -> float:
    model.train()
    losses = AverageMeter()

    for images, targets in loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad(set_to_none=True)

        if scaler is not None:
            with torch.cuda.amp.autocast():
                loss_dict = model(images, targets)
                loss = sum(loss_dict.values())
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss_dict = model(images, targets)
            loss = sum(loss_dict.values())
            loss.backward()
            optimizer.step()

        losses.update(loss.item(), n=len(images))

    return float(losses.avg)

@torch.inference_mode()
def val_loss(model, loader, device) -> float:
    # torchvision detection returns losses only in train mode
    model.train()
    losses = AverageMeter()

    for images, targets in loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        loss = sum(loss_dict.values())
        losses.update(loss.item(), n=len(images))

    return float(losses.avg)

def fit_frcnn(
    model,
    train_loader,
    val_loader,
    device: str,
    epochs: int,
    patience: int,
    ckpt_path,
    lr: float = 1e-4,
    mixed_precision: bool = True,
):
    device = torch.device(device)
    model.to(device)

    optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=lr)
    scaler = torch.cuda.amp.GradScaler() if (mixed_precision and device.type == "cuda") else None

    best = float("inf")
    bad = 0
    history: List[Dict[str, float]] = []

    with Timer() as t_train:
        for epoch in range(1, epochs + 1):
            tr = train_one_epoch(model, train_loader, optimizer, device, scaler)
            va = val_loss(model, val_loader, device)
            history.append({"epoch": epoch, "train_loss": tr, "val_loss": va})

            # checkpoint best
            if va < best - 1e-4:
                best = va
                bad = 0
                save_checkpoint(ckpt_path, epoch, model, optimizer, extra={"best_val_loss": best})
            else:
                bad += 1
                if bad >= patience:
                    break

    return model, history, t_train.seconds

Writing src/train.py


In [12]:
%%writefile src/eval.py
from typing import Dict, List
import torch

from src.metrics import map50, precision_recall_iou50
from src.utils import Timer

@torch.inference_mode()
def eval_frcnn(model, test_loader, device: str) -> Dict[str, float]:
    device = torch.device(device)
    model.to(device)
    model.eval()

    preds: List[Dict[str, torch.Tensor]] = []
    targets: List[Dict[str, torch.Tensor]] = []

    for images, targs in test_loader:
        images = [img.to(device) for img in images]
        out = model(images)[0]
        preds.append({k: v.detach().cpu() for k, v in out.items()})
        targets.append({k: v.detach().cpu() for k, v in targs[0].items()})

    m = map50(preds, targets)
    p, r = precision_recall_iou50(preds, targets, score_thresh=0.5)

    # Inference speed (images/sec), measure on up to 50 samples
    n_measure = min(50, len(test_loader.dataset))
    # warmup
    dummy = torch.zeros((3, 512, 512), device=device)
    _ = model([dummy])
    _ = model([dummy])

    with Timer() as t:
        for i in range(n_measure):
            img, _ = test_loader.dataset[i]
            _ = model([img.to(device)])
    ips = n_measure / max(t.seconds, 1e-9)

    return {
        "mAP@0.5": float(m),
        "precision": float(p),
        "recall": float(r),
        "inference_img_per_sec": float(ips),
    }

Writing src/eval.py


In [13]:
%%writefile src/yolo_utils.py
from pathlib import Path
from typing import Dict, List, Tuple
import yaml
import torch
from PIL import Image

def xyxy_to_yolo(box_xyxy: torch.Tensor, w: int, h: int) -> Tuple[float, float, float, float]:
    x1, y1, x2, y2 = box_xyxy.tolist()
    xc = ((x1 + x2) / 2.0) / w
    yc = ((y1 + y2) / 2.0) / h
    bw = (x2 - x1) / w
    bh = (y2 - y1) / h
    return xc, yc, bw, bh

def export_to_yolo(dataset, split: Dict[str, List[int]], out_root: Path, class_names: List[str]) -> Path:
    """
    dataset[idx] -> (img_tensor [3,512,512] float 0..1, target dict boxes xyxy, labels 1..K)
    YOLO labels must be 0..K-1
    """
    out_root.mkdir(parents=True, exist_ok=True)

    for s in ["train", "val", "test"]:
        (out_root / "images" / s).mkdir(parents=True, exist_ok=True)
        (out_root / "labels" / s).mkdir(parents=True, exist_ok=True)

        for new_i, idx in enumerate(split[s]):
            img, target = dataset[idx]
            H, W = img.shape[-2], img.shape[-1]

            img_pil = Image.fromarray((img.permute(1,2,0).numpy() * 255).astype("uint8"))
            img_path = out_root / "images" / s / f"{new_i:06d}.jpg"
            img_pil.save(img_path, quality=95)

            label_path = out_root / "labels" / s / f"{new_i:06d}.txt"
            lines = []
            for box, lab in zip(target["boxes"], target["labels"]):
                cls = int(lab.item()) - 1
                xc, yc, bw, bh = xyxy_to_yolo(box, W, H)
                lines.append(f"{cls} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")
            label_path.write_text("\n".join(lines))

    data_yaml = {
        "path": str(out_root),
        "train": "images/train",
        "val": "images/val",
        "test": "images/test",
        "names": class_names,
    }
    yaml_path = out_root / "data.yaml"
    yaml_path.write_text(yaml.safe_dump(data_yaml, sort_keys=False))
    return yaml_path

Writing src/yolo_utils.py


In [14]:
%%writefile src/run_yolo.py
from pathlib import Path
from typing import Dict
import json

from ultralytics import YOLO

def run_yolov8n(data_yaml: Path, out_dir: Path, exp_name: str, epochs: int, imgsz: int, batch: int) -> Dict:
    out_dir.mkdir(parents=True, exist_ok=True)

    model = YOLO("yolov8n.pt")  # pretrained nano model (transfer learning)

    model.train(
        data=str(data_yaml),
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        project=str(out_dir),
        name=exp_name,
        verbose=False,
    )

    val = model.val(data=str(data_yaml), imgsz=imgsz, batch=1, verbose=False)

    results = {
        "mAP@0.5": float(val.box.map50),
        "precision": float(val.box.mp),
        "recall": float(val.box.mr),
        "speed_ms_per_img": dict(val.speed),  # preprocess/inference/postprocess
    }
    return results

Writing src/run_yolo.py


In [15]:
!touch src/__init__.py

In [16]:
from pathlib import Path
import torch
from torch.utils.data import DataLoader, Subset

from src.config import Config
from src.utils import set_seed, split_indices, save_json
from src.datasets import PennFudanDetection, PetsSubsetDetection, collate_fn
from src.model import build_frcnn
from src.train import fit_frcnn
from src.eval import eval_frcnn
from src.yolo_utils import export_to_yolo
from src.run_yolo import run_yolov8n

cfg = Config()
set_seed(cfg.seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


'cuda'

In [17]:
penn = PennFudanDetection(root=str(cfg.data_dir / "pennfudan"), img_size=cfg.img_size)

penn_split = split_indices(len(penn), cfg.split_train, cfg.split_val, cfg.split_test, seed=cfg.seed)
save_json(penn_split, cfg.splits_dir / "pennfudan_split.json")

penn_train = Subset(penn, penn_split["train"])
penn_val   = Subset(penn, penn_split["val"])
penn_test  = Subset(penn, penn_split["test"])

penn_train_loader = DataLoader(penn_train, batch_size=cfg.frcnn_batch_size, shuffle=True, num_workers=cfg.num_workers, collate_fn=collate_fn)
penn_val_loader   = DataLoader(penn_val, batch_size=cfg.frcnn_batch_size, shuffle=False, num_workers=cfg.num_workers, collate_fn=collate_fn)
penn_test_loader  = DataLoader(penn_test, batch_size=1, shuffle=False, num_workers=cfg.num_workers, collate_fn=collate_fn)

len(penn), len(penn_train), len(penn_val), len(penn_test)

(170, 118, 25, 27)

In [18]:
# Penn-Fudan: 1 class (person) + background => num_classes = 2
frcnn_penn = build_frcnn(num_classes=2)

ckpt_path = cfg.ckpt_dir / "frcnn_penn_best.pth"
frcnn_penn, penn_history, penn_train_time = fit_frcnn(
    frcnn_penn,
    penn_train_loader,
    penn_val_loader,
    device=device,
    epochs=cfg.penn_epochs,       # 10–15
    patience=cfg.patience,        # early stopping allowed
    ckpt_path=ckpt_path,
    lr=1e-4,
    mixed_precision=True
)

penn_train_time, len(penn_history)

Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth


100%|██████████| 74.2M/74.2M [00:00<00:00, 176MB/s]
/content/assignment_2/src/train.py:63: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if (mixed_precision and device.type == "cuda") else None
/content/assignment_2/src/train.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


(45.18125667700002, 4)

In [19]:
penn_frcnn_metrics = eval_frcnn(frcnn_penn, penn_test_loader, device=device)

penn_frcnn_results = {
    "dataset": "Penn-Fudan",
    "model": "Faster R-CNN MobileNetV3 FPN",
    **penn_frcnn_metrics,
    "training_time_sec": penn_train_time,
}
save_json(penn_frcnn_results, cfg.outputs_dir / "penn_frcnn_results.json")
penn_frcnn_results

{'dataset': 'Penn-Fudan',
 'model': 'Faster R-CNN MobileNetV3 FPN',
 'mAP@0.5': 0.9740066528320312,
 'precision': 0.8450704225233088,
 'recall': 0.9836065573609245,
 'inference_img_per_sec': 23.073146693030765,
 'training_time_sec': 45.18125667700002}

In [20]:
penn_yaml = export_to_yolo(
    dataset=penn,
    split=penn_split,
    out_root=cfg.data_dir / "yolo_pennfudan",
    class_names=["person"],
)

penn_yolo_metrics = run_yolov8n(
    data_yaml=penn_yaml,
    out_dir=cfg.outputs_dir,
    exp_name="penn_yolov8n",
    epochs=cfg.penn_epochs,   # 10–15
    imgsz=cfg.img_size,       # 512
    batch=cfg.yolo_batch_size # 8–16
)

penn_yolo_results = {
    "dataset": "Penn-Fudan",
    "model": "YOLOv8n",
    **penn_yolo_metrics,
}
save_json(penn_yolo_results, cfg.outputs_dir / "penn_yolo_results.json")
penn_yolo_results

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/yolo_pennfudan/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=12, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=penn_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

{'dataset': 'Penn-Fudan',
 'model': 'YOLOv8n',
 'mAP@0.5': 0.970297931144147,
 'precision': 0.9713366614963532,
 'recall': 0.8802342534245847,
 'speed_ms_per_img': {'preprocess': 1.0294885200073622,
  'inference': 10.805430399987017,
  'loss': 0.006472560007750872,
  'postprocess': 1.742720679994818}}

In [21]:
breeds = [
    "Abyssinian",
    "Bengal",
    "Birman",
    "Bombay",
    "British_Shorthair",
    "Egyptian_Mau",
    "Maine_Coon",
    "Persian"
]

In [22]:
pets = PetsSubsetDetection(root=str(cfg.data_dir / "pets"), breeds=breeds, img_size=cfg.img_size)

pets_split = split_indices(len(pets), cfg.split_train, cfg.split_val, cfg.split_test, seed=cfg.seed)
save_json(pets_split, cfg.splits_dir / "pets_split.json")

pets_train = Subset(pets, pets_split["train"])
pets_val   = Subset(pets, pets_split["val"])
pets_test  = Subset(pets, pets_split["test"])

pets_train_loader = DataLoader(pets_train, batch_size=cfg.frcnn_batch_size, shuffle=True, num_workers=cfg.num_workers, collate_fn=collate_fn)
pets_val_loader   = DataLoader(pets_val, batch_size=cfg.frcnn_batch_size, shuffle=False, num_workers=cfg.num_workers, collate_fn=collate_fn)
pets_test_loader  = DataLoader(pets_test, batch_size=1, shuffle=False, num_workers=cfg.num_workers, collate_fn=collate_fn)

len(pets), len(pets_train), len(pets_val), len(pets_test)

(789, 552, 118, 119)

In [23]:
# Pets: K breeds + background => num_classes = K + 1
frcnn_pets = build_frcnn(num_classes=len(breeds) + 1)

ckpt_path = cfg.ckpt_dir / "frcnn_pets_best.pth"
frcnn_pets, pets_history, pets_train_time = fit_frcnn(
    frcnn_pets,
    pets_train_loader,
    pets_val_loader,
    device=device,
    epochs=cfg.pets_epochs,     # 15–20
    patience=cfg.patience,
    ckpt_path=ckpt_path,
    lr=1e-4,
    mixed_precision=True
)

pets_train_time, len(pets_history)

/content/assignment_2/src/train.py:63: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if (mixed_precision and device.type == "cuda") else None
/content/assignment_2/src/train.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


(187.86921026599998, 7)

In [24]:
pets_frcnn_metrics = eval_frcnn(frcnn_pets, pets_test_loader, device=device)

pets_frcnn_results = {
    "dataset": "Oxford-IIIT Pets (subset)",
    "model": "Faster R-CNN MobileNetV3 FPN",
    **pets_frcnn_metrics,
    "training_time_sec": pets_train_time,
}
save_json(pets_frcnn_results, cfg.outputs_dir / "pets_frcnn_results.json")
pets_frcnn_results

{'dataset': 'Oxford-IIIT Pets (subset)',
 'model': 'Faster R-CNN MobileNetV3 FPN',
 'mAP@0.5': 0.8659413456916809,
 'precision': 0.7835820895463912,
 'recall': 0.8823529411690558,
 'inference_img_per_sec': 35.13944216516485,
 'training_time_sec': 187.86921026599998}

In [25]:
pets_yaml = export_to_yolo(
    dataset=pets,
    split=pets_split,
    out_root=cfg.data_dir / "yolo_pets",
    class_names=breeds,
)

pets_yolo_metrics = run_yolov8n(
    data_yaml=pets_yaml,
    out_dir=cfg.outputs_dir,
    exp_name="pets_yolov8n",
    epochs=cfg.pets_epochs,   # 15–20
    imgsz=cfg.img_size,       # 512
    batch=cfg.yolo_batch_size # 8–16
)

pets_yolo_results = {
    "dataset": "Oxford-IIIT Pets (subset)",
    "model": "YOLOv8n",
    **pets_yolo_metrics,
}
save_json(pets_yolo_results, cfg.outputs_dir / "pets_yolo_results.json")
pets_yolo_results

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/yolo_pets/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=18, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=pets_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspectiv

{'dataset': 'Oxford-IIIT Pets (subset)',
 'model': 'YOLOv8n',
 'mAP@0.5': 0.8768464422029985,
 'precision': 0.8285325843848361,
 'recall': 0.8892983911613251,
 'speed_ms_per_img': {'preprocess': 1.0118702627042402,
  'inference': 8.514025822033176,
  'loss': 0.006827457630575829,
  'postprocess': 1.306927008476647}}

In [26]:
import pandas as pd
from src.utils import load_json

rows = [
    load_json(cfg.outputs_dir / "penn_frcnn_results.json"),
    load_json(cfg.outputs_dir / "penn_yolo_results.json"),
    load_json(cfg.outputs_dir / "pets_frcnn_results.json"),
    load_json(cfg.outputs_dir / "pets_yolo_results.json"),
]
df = pd.DataFrame(rows)

# Match assignment table columns
cols = ["dataset", "model", "mAP@0.5", "precision", "recall", "training_time_sec", "inference_img_per_sec", "speed_ms_per_img"]
df.reindex(columns=cols)

,dataset,model,mAP@0.5,precision,recall,training_time_sec,inference_img_per_sec,speed_ms_per_img
0,Penn-Fudan,Faster R-CNN MobileNetV3 FPN,0.974007,0.845070,0.983607,45.181257,23.073147,NaN
1,Penn-Fudan,YOLOv8n,0.970298,0.971337,0.880234,NaN,NaN,"{'preprocess': 1.0294885200073622, 'inference'..."
2,Oxford-IIIT Pets (subset),Faster R-CNN MobileNetV3 FPN,0.865941,0.783582,0.882353,187.869210,35.139442,NaN
3,Oxford-IIIT Pets (subset),YOLOv8n,0.876846,0.828533,0.889298,NaN,NaN,"{'preprocess': 1.0118702627042402, 'inference'..."


In [27]:
import json
import pandas as pd
from pathlib import Path

out = Path("outputs")
files = [
    "penn_frcnn_results.json",
    "penn_yolo_results.json",
    "pets_frcnn_results.json",
    "pets_yolo_results.json",
]

rows = []
for f in files:
    d = json.loads((out / f).read_text())
    rows.append({
        "dataset": d.get("dataset"),
        "model": d.get("model"),
        "mAP@0.5": d.get("mAP@0.5"),
        "precision": d.get("precision"),
        "recall": d.get("recall"),
        "inference_speed": d.get("inference_img_per_sec", None),  # FRCNN uses img/sec
        "inference_ms": d.get("speed_ms_per_img", {}).get("inference", None),  # YOLO uses ms/img
    })

df = pd.DataFrame(rows)
df

,dataset,model,mAP@0.5,precision,recall,inference_speed,inference_ms
0,Penn-Fudan,Faster R-CNN MobileNetV3 FPN,0.974007,0.845070,0.983607,23.073147,NaN
1,Penn-Fudan,YOLOv8n,0.970298,0.971337,0.880234,NaN,10.805430
2,Oxford-IIIT Pets (subset),Faster R-CNN MobileNetV3 FPN,0.865941,0.783582,0.882353,35.139442,NaN
3,Oxford-IIIT Pets (subset),YOLOv8n,0.876846,0.828533,0.889298,NaN,8.514026


In [28]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
from pathlib import Path
import random

def visualize_frcnn_predictions(model, dataset_subset, device, save_dir, num_images=4, score_thresh=0.5):
    model.eval()
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    indices = random.sample(range(len(dataset_subset)), num_images)

    for i, idx in enumerate(indices):
        img, target = dataset_subset[idx]

        with torch.no_grad():
            pred = model([img.to(device)])[0]

        img_np = img.permute(1, 2, 0).cpu().numpy()

        fig, ax = plt.subplots(1, figsize=(6,6))
        ax.imshow(img_np)

        # 🔵 Ground truth boxes (green)
        for box in target["boxes"]:
            x1, y1, x2, y2 = box
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=2, edgecolor="lime", facecolor="none"
            )
            ax.add_patch(rect)

        # 🔴 Predicted boxes (red)
        for box, score in zip(pred["boxes"], pred["scores"]):
            if score < score_thresh:
                continue
            x1, y1, x2, y2 = box.cpu()
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=2, edgecolor="red", facecolor="none"
            )
            ax.add_patch(rect)

        ax.set_title(f"Prediction {i+1}")
        ax.axis("off")

        out_path = save_dir / f"frcnn_pred_{i+1}.png"
        plt.savefig(out_path, bbox_inches="tight", dpi=200)
        plt.close()

    print(f"✅ Saved {num_images} prediction images to {save_dir}")

In [29]:
visualize_frcnn_predictions(
    frcnn_penn,
    penn_test,
    device,
    cfg.outputs_dir / "examples_penn_frcnn",
    num_images=4
)

✅ Saved 4 prediction images to outputs/examples_penn_frcnn


In [30]:
visualize_frcnn_predictions(
    frcnn_pets,
    pets_test,
    device,
    cfg.outputs_dir / "examples_pets_frcnn",
    num_images=4
)


✅ Saved 4 prediction images to outputs/examples_pets_frcnn


In [31]:
import pandas as pd
from pathlib import Path

# ---- PETS YOLO ----
pets_csv = Path("runs/detect/outputs/pets_yolov8n/results.csv")
pets_df = pd.read_csv(pets_csv)

# total training time (seconds)
pets_train_time = pets_df["time"].sum()

print("Pets YOLO training time (sec):", pets_train_time)

Pets YOLO training time (sec): 1715.0281


In [32]:
penn_csv = Path("runs/detect/outputs/penn_yolov8n/results.csv")
penn_df = pd.read_csv(penn_csv)

penn_train_time = penn_df["time"].sum()

print("Penn YOLO training time (sec):", penn_train_time)

Penn YOLO training time (sec): 231.33816000000002


In [33]:
# example
penn_yolo_results["training_time_sec"] = penn_train_time
pets_yolo_results["training_time_sec"] = pets_train_time

In [34]:
save_json(penn_yolo_results, cfg.outputs_dir / "penn_yolo_results.json")
save_json(pets_yolo_results, cfg.outputs_dir / "pets_yolo_results.json")

In [ ]:
!zip -r assignment2_results.zip /content/assignment_2

Streaming output truncated to the last 5000 lines.
  adding: content/assignment_2/data/pets/annotations/trimaps/._Egyptian_Mau_157.png (deflated 36%)
  adding: content/assignment_2/data/pets/annotations/trimaps/._boxer_146.png (deflated 36%)
  adding: content/assignment_2/data/pets/annotations/trimaps/._chihuahua_20.png (deflated 36%)
  adding: content/assignment_2/data/pets/annotations/trimaps/._British_Shorthair_157.png (deflated 36%)
  adding: content/assignment_2/data/pets/annotations/trimaps/._Russian_Blue_49.png (deflated 36%)
  adding: content/assignment_2/data/pets/annotations/trimaps/Russian_Blue_28.png (deflated 24%)
  adding: content/assignment_2/data/pets/annotations/trimaps/english_cocker_spaniel_21.png (deflated 3%)
  adding: content/assignment_2/data/pets/annotations/trimaps/._american_pit_bull_terrier_19.png (deflated 36%)
  adding: content/assignment_2/data/pets/annotations/trimaps/leonberger_27.png (deflated 4%)
  adding: content/assignment_2/data/pets/annotations/tri

In [ ]:
from google.colab import files
files.download('assignment2_results.zip')